In [ ]:
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from random import randint
import glob
import os
import random

from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


import torchvision
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset, TensorDataset
import torch.optim as optim

%matplotlib inline

In [ ]:
class CamaraModelDataset(Dataset):

    def __init__(self, path2data, labels, transforms=None, is_train = False):
        

        self.transform = transforms 
        
        self.X= []
        self.y = []
        for key, value in labels.items():
            path_folder = os.path.join(path, value, '*.jpg')
            files = glob.glob(path_folder) 
            self.X.extend(files)
            self.y.extend(int(key)*np.ones(len(files)))

            
    def __getitem__(self, index):
        
        path = self.X[index]
        label = self.y[index]

        with open(path, 'rb') as f:
            flbase = os.path.basename(path)
            
            with Image.open(f) as img:
                 image = img.convert('RGB')
                    
        if self.transform is not None:
            image = self.transform(image)

        return image, label

    def __len__(self):
        return len(self.X)

In [ ]:
class Image2Vector(object):

    def __call__(self, image):

        image = np.asarray(image)
        #image = np.reshape(image, (image.shape[0]*image.shape[1], image.shape[2]))

        return image

In [ ]:
img_width = img_height=128

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((img_width, img_height)),
        Image2Vector()
    ])
}

In [ ]:
labels = {1:'dandelion', 2:'tulip', 3:'sunflower', 4:'daisy', 5:'rose'}
path = 'data/flowers/'

In [ ]:
dsets = {'train': CamaraModelDataset(path, labels, transforms=data_transforms['train'],  is_train = True)}

In [ ]:
def CamaraPhotos(dsets, labels):
    
    rand_img = random.randrange(0, len(dsets))
    img = dsets[rand_img][0]
    y = dsets[rand_img][1]
    plt.imshow(img)
    plt.title('Label: {}'.format(labels[y]))

    plt.axis('off')

In [ ]:
plt.figure(figsize=(20,5))
for i in range(0, 3):
    plt.subplot(1,3,i+1)

    CamaraPhotos(dsets['train'], labels)

In [ ]:
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)
    
    def forward(self, x):
        out = self.linear(x)
        return out

In [ ]:
input_size = img_width*img_height
num_classes = len(labels)

logisticRegression = LogisticRegression(input_size, num_classes)

In [ ]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, img_width*img_height))
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch: [%d/%d], Step: [%d/%d], Loss: %.4f' 
                   % (epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))


In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Turn up tolerance for faster convergence
logisticRegr = LogisticRegression()
logisticRegr.fit(X_train, y_train)

In [ ]:
predictions = logisticRegr.predict(X_test)

In [ ]:
score = logisticRegr.score(X_test, y_test)
print(score)

In [ ]:
index = 0
misclassifiedIndexes = []
for label, predict in zip(y_test, predictions):
    if label != predict: 
        misclassifiedIndexes.append(index)
        index +=1

In [ ]:
plt.figure(figsize=(20,4))
for plotIndex, badIndex in enumerate(misclassifiedIndexes[0:5]):
    plt.subplot(1, 5, plotIndex + 1)
    plt.imshow(np.reshape(X_test[badIndex], (20,20)), cmap=plt.cm.gray)
    plt.axis('off')
    plt.title('Predicted: {}, Actual: {}'.format(predictions[badIndex], y_test[badIndex]), fontsize = 15)